In [4]:
import os
import shutil
import subprocess
import numpy as np

def run_geant4_simulation(mineral_name, particle_type, n_events, energies_gev):
    """
    Dynamically creates a Geant4 macro file and runs the simulation for a given list of energies.

    Args:
        mineral_name (str): Name of the target material (must be defined in Geant4).
        particle_type (str): Particle type for the gun (e.g., "mu-", "proton").
        n_events (int): Number of primary particles to simulate per energy.
        energies_gev (np.array): Array of energies in GeV to simulate.
    """

    default_output_dir = "./build/output"
    final_output_dir = f"./{mineral_name}_{particle_type}"
    
    if not os.path.exists(default_output_dir):
        os.makedirs(default_output_dir)
    
    if not os.path.exists(final_output_dir):
        os.makedirs(final_output_dir)

    macro_content = f"""# Auto-generated macro file
/control/verbose 0
/run/verbose 1
/tracking/verbose 0

# --- Geometry Setup ---
/testhadr/TargetMat        {mineral_name}
/testhadr/TargetRadius     10 m
/testhadr/TargetLength     200 m

# --- Physics ---
/run/setCut                0.005 mm
/run/initialize

# --- Run Loop ---
/gun/particle {particle_type}
"""

    for energy in energies_gev:
        macro_content += f"\n/gun/energy {energy:.16f} GeV\n"
        macro_content += f"/run/beamOn {n_events}\n"

    macro_filename = "./build/run.in"
    with open(macro_filename, "w") as f:
        f.write(macro_content)

    executable = "./build/Run"
    
    if not os.path.exists(executable):
        print(f"Error: Executable '{executable}' not found. Compile the C++ code first.")
        return

    print(f"Starting Geant4 simulation for {mineral_name} with {len(energies_gev)} energy points...")
    
    try:
        result = subprocess.run([executable, macro_filename], check=True, capture_output=True, text=True)
        print(result.stdout)
        
    except subprocess.CalledProcessError as e:
        print("Error running Geant4:")
        print(e.stderr)
        return
    
    print(f"Simulation finished. Moving files to {final_output_dir}...")
    
    for i, energy in enumerate(energies_gev):
        src_filename = f"outNuclei_{i}.txt"
        src_path = os.path.join(default_output_dir, src_filename)
        
        dst_filename = f"outNuclei_{energy:.6f}.txt"
        dst_path = os.path.join(final_output_dir, dst_filename)
        
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
        else:
            print(f"Warning: Expected output file {src_filename} not found.")

    print("All done.")


In [ ]:
energies = np.logspace(-3, 4, 100)

In [ ]:
run_geant4_simulation("Olivine", "neutron", 10000, energies)

Starting Geant4 simulation for StdRock with 10 energy points...

**************************************************************
 Geant4 version Name: geant4-11-03-patch-02 [MT]   (25-April-2025)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************

### Target material set to StdRock

========= Table of registered couples ============================

Index : 0     used in the geometry : Yes
 Material : G4_Galactic
 Range cuts        :  gamma  5 um     e-  5 um     e+  5 um  proton 5 um 
 Energy thresholds :  gamma  990 eV     e-  990 eV     e+  990 eV  proton 500 eV 
 Region(s) which use this couple : 
    DefaultRegionForTheWorld

Index : 1     used in the geometry : Yes
 Material : St